# REICH

Two copies of a single part composer (generator) produces a massively abbreviated version of Steve Reich's Piano Phase. 

Running this notebook requires a jupyter kernel that contains the musx package. See [INSTALL.md](https://github.com/musx-admin/musx/blob/main/INSTALL.md) for directions on how to install musx in your environment.
<hr style="height:1px; color: grey;">

Python imports:

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from musx import version, Note, Seq, Score, MidiFile, Cycle, keynum, playfile, setmidiplayer
print(f"musx version: {version}")

This notebook generates MIDI files and automatically plays them using [fluidsynth](https://www.fluidsynth.org/download/) and the [MuseScore_General.sf3](https://ftp.osuosl.org/pub/musescore/soundfont/MuseScore_General) sound font. See [INSTALL.md](https://github.com/musx-admin/musx/blob/main/INSTALL.md) for how to install a terminal-based MIDI player to use with musx.  If you don't have a player installed you can access the output files in the same directory as this notebook:

In [ ]:
setmidiplayer("fluidsynth -iq -g1 /usr/local/sf/MuseScore_General.sf2")
print('OK!')

The `piano_phase()` generator produces the melodic material for both pianos:

In [ ]:
def piano_phase(score, end, keys, rate):
    """
    Composes a piano part for Steve Reich's Piano Phase.

    Parameters
    ----------
    score : Score
        The scheduling queue to run the composer in.
    end : int | float
        The total duration of the piece.
    keys : list
        A list of midi key numbers to play in a loop.
    rate : int | float
        The rhythm to use.    
    """
    # Create a cyclic pattern to produce the key numbers.
    pattern = Cycle(keys)
    # Generate notes until score time is >= end.
    while score.now < end:
        # Get the next key number.
        knum = pattern.next()
        # Create a midi note to play it.
        note = Note(time=score.now, duration=rate, pitch=knum, amplitude=.9)
        # Add the midi note to the score.
        score.add(note)
        # Return the amount of time until this composer runs again.
        yield rate
print(f"piano_phase: {piano_phase}")

Define track0 to be a midi meta track that holds tempo, midi instrument assignments, micro tuning, etc.:

In [ ]:
track0 = MidiFile.metatrack()
print(f"track0: {track0}")

Track1 will hold the composition:

In [ ]:
track1 = Seq()
print(f"track1: {track1}")

Create a score and pass it track1 to hold the output midi event data:

In [ ]:
score = Score(out=track1)
print(f"score: {score}")

Convert Reich's piano notes into a list of midi key numbers:

In [ ]:
keys = keynum("e4 f# b c#5 d f#4 e c#5 b4 f# d5 c#")
print(f"keys: {keys}")

Create two instances of the `piano_phase()` composer and run them at slightly different rates to cause Reich's phasing effect:

In [ ]:
pianos = [piano_phase(score, 20, keys, .167), 
          piano_phase(score, 20, keys, .170)]
print(f"pianos: {pianos}")

Compose the composition:

In [ ]:
score.compose(pianos)
print("OK!")

Write the tracks to a midi file in the current directory and play it if possible:

In [ ]:
file = MidiFile("reich.mid", [track0, track1]).write()
print(f"Wrote '{file.pathname}'")
playfile(file.pathname)